In [1]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedTokenizerFast


# Download configuration from huggingface.co and cache.
# model_id = "facebook/mbart-large-50-many-to-many-mmt"
# model_id = "google-t5/t5-base"

# does not download the pretrained weights, just affects configuration
# use AutoModelForSeq2SeqLM.from_pretrained to also download the weights
# config = AutoConfig.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_config(config)

In [191]:
# now, need to add a tokenizer and embedding layer to the top of the model
auto_tk = AutoTokenizer.from_pretrained(model_id)

c:\Users\abhiv\miniconda3\envs\mse244\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [197]:
print(auto_tk.vocab_size)
print(list(auto_tk.get_vocab())[:100])

32100
['”', '▁punct', '▁taken', 'elecommunication', '▁cables', '▁helpful', 'cita', 'gasesc', '▁buyer', 'aide', '▁bisher', '▁upgrading', '▁Haftung', '▁crunchy', '▁Colegi', '▁Bollywood', '▁historical', '▁NASA', '▁Minute', 'teamed', '▁peace', '▁Diesel', '▁--', 'gate', '▁zip', '▁zuständig', '▁define', '▁Digi', '▁Diversity', '▁engage', '▁peninsula', 'moni', '▁phone', '▁reflecting', '▁experienta', 'blin', '▁poems', 'zugleich', '▁force', 'brücke', 'Lib', 'cul', 'ministerium', '▁song', '▁themes', '▁suis', 'admi', 'gesagt', 'â', 'OC', '▁birouri', '▁activités', '▁franchi', '▁Cushion', '▁Versand', '▁mittels', '▁strig', '▁diffusion', 'lebt', '▁payment', '▁crashes', '▁Qualcomm', '▁Strange', 'lov', '▁nächste', '▁Januar', '▁bestellen', '▁Sat', '▁aplicat', '▁revolution', 'soluble', '▁legend', 'terribly', '▁fitted', '▁run', '▁everyone', '▁ramp', '▁Fotos', 'absorbed', '▁ignor', '▁Bangalore', '▁Commissioner', 'ani', '▁funnel', '▁Proceedings', 'erweise', 'tech', '▁GREAT', '▁dilemma', 'identifying', '▁("',

In [2]:
from tokenizers import Tokenizer, decoders
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, ByteLevel
from datasets import load_dataset

# dataset = load_dataset("csv", data_files="data/v3.csv")

# dataset = load_dataset("csv", data_files="data/v4.csv")
# data_splits = dataset['train'].train_test_split(0.1)
# train, test = data_splits['train'], data_splits['test']

In [3]:
# train.to_csv('./data/train.csv')
# test.to_csv('./data/test.csv')
train = load_dataset('csv', data_files='data/train.csv')['train']
test = load_dataset('csv', data_files='data/test.csv')['train']

In [63]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
# try to restrict vocab size to be similar to our custom tokenizer vocab
CUSTOM_VOCAB_SIZE = 7150
trainer = BpeTrainer(vocab_size=CUSTOM_VOCAB_SIZE, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])


obolo_data = train['Obolo']

tokenizer.pre_tokenizer = ByteLevel()
tokenizer.decoder = decoders.ByteLevel()
tokenizer.train_from_iterator(obolo_data, trainer)
print(tokenizer.get_vocab_size())
tokenizer.save('./custom_tokenizers/obolo-bpe-tokenizer-small.json')

7150


In [5]:
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
# try to restrict vocab size to be similar to our custom tokenizer vocab
CUSTOM_VOCAB_SIZE = 7150
trainer = WordPieceTrainer(vocab_size=CUSTOM_VOCAB_SIZE, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])


obolo_data = train['Obolo']

tokenizer.pre_tokenizer = Whitespace()
tokenizer.decoder = decoders.WordPiece()
tokenizer.train_from_iterator(obolo_data, trainer)
print(tokenizer.get_vocab_size())
tokenizer.save('./custom_tokenizers/obolo-wordpiece-tokenizer-small.json')

7150


In [61]:
import json 

obolo_tokenizer = PreTrainedTokenizerFast(tokenizer_file='./custom_tokenizers/obolo-bpe-tokenizer.json', padding='left')
print("Original BPE Tokenizer Length:", obolo_tokenizer.vocab_size)
test_sentence = "jeremot liyi luku sene kot"
print(obolo_tokenizer(test_sentence))
custom_token_set = dict(json.load( open( "./custom_tokenizers/custom_obolo_tokenizer_vocab_2.json" ) )).keys()
# Train a new tokenizer using the am_train and the old tokenizer object.
print(len(custom_token_set))
new_tokenizer = obolo_tokenizer.train_new_from_iterator(custom_token_set, vocab_size=100000)
print(len(set(new_tokenizer.vocab).intersection(set(obolo_tokenizer.vocab))))
obolo_tokenizer.add_tokens(list(new_tokenizer.vocab))

print(new_tokenizer(test_sentence))
print(obolo_tokenizer(test_sentence))

print(new_tokenizer.vocab_size)
print(len(new_tokenizer))
print(obolo_tokenizer.vocab_size)
print(len(obolo_tokenizer))


Original BPE Tokenizer Length: 17596
{'input_ids': [5750, 610, 66, 2751, 35, 516, 178, 61, 233], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
7150
7302
{'input_ids': [6936, 513, 296, 225, 2363, 3933, 347, 358], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [89, 1087, 233, 42, 185, 66, 42, 18623, 42, 9386, 42, 1030], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
8343
8343
17596
18637


In [60]:
tokenizer_small = PreTrainedTokenizerFast(tokenizer_file='./custom_tokenizers/obolo-bpe-tokenizer-small.json', padding='left')
print(len(tokenizer_small))

6864


In [45]:
# obolo_tokenizer.save_pretrained(save_directory='./custom_tokenizers/custom_plus_bpe_obolo_tokenizer')
obolo_tokenizer = PreTrainedTokenizerFast(tokenizer_file='./custom_tokenizers/custom_plus_bpe_obolo_tokenizer/tokenizer.json', padding='left')
print(len(obolo_tokenizer))

18887


In [30]:
# find total words and unique words in the Obolo set
import string
data = train['Obolo']+test['Obolo']
unique_words_verse = [set(s.translate(str.maketrans('', '', string.punctuation)).strip().split()) for s in data]
unique_words = set()
for s in unique_words_verse:
    unique_words.update(s)
print(len(unique_words))

21988


In [32]:
from collections import defaultdict
freq = defaultdict(int)
for s in data:
    words = s.translate(str.maketrans('', '', string.punctuation)).strip().split()
    for w in words:
        freq[w] += 1

In [33]:
count = 0
for k, v in freq.items():
    if v == 1:
        count += 1
count

8865

In [36]:
total = sum(freq.values()) 
total

882513

In [38]:
print(tokenizer.get_vocab_size())
unique_words_verse = [set(verse.split()) for verse in train['Obolo']]
unique_words = set()
for s in unique_words_verse:
    unique_words.update(s)
print(len(unique_words))

17596
30600


In [39]:
encoded_inputs = tokenizer.encode_batch(obolo_data[:5])
for idx, enc in enumerate(encoded_inputs):
    print(obolo_data[idx])
    print(enc.tokens)

eya orọ, nnenikana isi lek ogwukaan̄-ene awaji, mè ito ito inyi, mè iriaak uriaak, mè ibeek mbeek, mè ijin otu, mè itap ofọnti ufieejit, mè ifuun̄ lek me ntọn̄.
['Ġeya', 'Ġorá»į', ',', 'Ġnnenikana', 'Ġisi', 'Ġlek', 'Ġogwukaan', 'ÌĦ-', 'ene', 'Ġawaji', ',', 'Ġme', 'ÌĢ', 'Ġito', 'Ġito', 'Ġinyi', ',', 'Ġme', 'ÌĢ', 'Ġiriaak', 'Ġuriaak', ',', 'Ġme', 'ÌĢ', 'Ġibeek', 'Ġmbeek', ',', 'Ġme', 'ÌĢ', 'Ġijin', 'Ġotu', ',', 'Ġme', 'ÌĢ', 'Ġitap', 'Ġofá»įnti', 'Ġufieejit', ',', 'Ġme', 'ÌĢ', 'Ġifuun', 'ÌĦ', 'Ġlek', 'Ġme', 'Ġntá»įn', 'ÌĦ.']
bọn ebirieen̄ kè kora ìre asiri mè elikana mè abiasafu. ebibi ere ebi otoko kè kora me ototun̄ ukan̄ me ototun̄ ukan̄.
['Ġbá»įn', 'Ġebirieen', 'ÌĦ', 'Ġke', 'ÌĢ', 'Ġkora', 'Ġi', 'ÌĢ', 're', 'Ġasiri', 'Ġme', 'ÌĢ', 'Ġelikana', 'Ġme', 'ÌĢ', 'Ġabiasafu', '.', 'Ġebibi', 'Ġere', 'Ġebi', 'Ġotoko', 'Ġke', 'ÌĢ', 'Ġkora', 'Ġme', 'Ġototun', 'ÌĦ', 'Ġukan', 'ÌĦ', 'Ġme', 'Ġototun', 'ÌĦ', 'Ġukan', 'ÌĦ.']
enyi mêchieen̄ ita me etete ebi ofifi ido. ijọn̄ ebi ochicha kinyi m

In [7]:
eng_tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

english_data = train['English']

eng_tokenizer.pre_tokenizer = ByteLevel()
eng_tokenizer.decoder = decoders.ByteLevel()
eng_tokenizer.train_from_iterator(english_data, trainer)

In [8]:
print(eng_tokenizer.get_vocab_size())
unique_words_verse = [set(verse.split()) for verse in english_data]
unique_words = set()
for s in unique_words_verse:
    unique_words.update(s)
print(len(unique_words))

18649
28148


In [167]:
encoded_inputs = tokenizer.encode_batch(english_data[:5])
for idx, enc in enumerate(encoded_inputs):
    print(english_data[idx])
    print(enc.tokens)

and he said unto him, we [are] passing from bethlehemjudah toward the side of mount ephraim; from thence [am] i: and i went to bethlehemjudah, but i [am now] going to the house of the lord; and there [is] no man that receiveth me to house.
['Ġand', 'Ġhe', 'Ġsaid', 'Ġunto', 'Ġhim', ',', 'Ġwe', 'Ġ[', 'are', ']', 'Ġpassing', 'Ġfrom', 'Ġbethlehemjudah', 'Ġtoward', 'Ġthe', 'Ġside', 'Ġof', 'Ġmount', 'Ġephraim', ';', 'Ġfrom', 'Ġthence', 'Ġ[', 'am', ']', 'Ġi', ':', 'Ġand', 'Ġi', 'Ġwent', 'Ġto', 'Ġbethlehemjudah', ',', 'Ġbut', 'Ġi', 'Ġ[', 'am', 'Ġnow', ']', 'Ġgoing', 'Ġto', 'Ġthe', 'Ġhouse', 'Ġof', 'Ġthe', 'Ġlord', ';', 'Ġand', 'Ġthere', 'Ġ[', 'is', ']', 'Ġno', 'Ġman', 'Ġthat', 'Ġreceiveth', 'Ġme', 'Ġto', 'Ġhouse', '.']
and ye shall appoint the possession of the city five thousand broad, and five and twenty thousand long, over against the oblation of the holy [portion]: it shall be for the whole house of israel.
['Ġand', 'Ġye', 'Ġshall', 'Ġappoint', 'Ġthe', 'Ġpossession', 'Ġof', 'Ġthe', 'Ġcity'

In [4]:
from custom_tokenizers.tokenizerv1 import parse_sentence
tokens = set()
for idx, s in enumerate(train['Obolo']):
    if idx % 10000 == 0:
        print(idx)
    tokens.update(set(parse_sentence(s)))
print('Custom tokenizer vocab size on train data:', len(tokens))

0
10000
20000
Custom tokenizer vocab size on train data: 6859


In [19]:
print(list(tokens)[:10])

['mônikpọ', 'baraken', 'môkilibi', 'nikanọ', 'ǹnenkwala', 'ri', 'magidala', 'mumumuk', 'pirisila', 'mêkiyiyeek']


In [16]:
sent = test['Obolo'][10]
print(sent)
print(parse_sentence(sent))
print(sent.replace(' ', ''))
print(''.join(parse_sentence(sent)))

mè initumu inyi samien ibe, “kpọ, mgbọ keyi owu ora ikan. nsabọn ebirieen̄ kwun̄ si kpekup kubọk owu okupbe. eya orọ, to ubọọn̄ nyi eji, ogwu òbokikpulu eji, mè ikisibi oyerebet inyi eji kubọk îkupbe me otutuuk ofifi ido geelek.”
['mè', 'i', 'ni', 'tumu', 'i', 'nyi', 'samien', 'i', 'be', ',', '“', 'kpọ', ',', 'm', 'gbọ', 'keyi', 'o', 'wu', 'o', 'ra', 'i', 'kan', '.', 'nsabọn', 'ebirieen̄', 'kwun̄', 'si', 'kpe', 'kup', 'kubọk', 'o', 'wu', 'o', 'kup', 'be', '.', 'e', 'ya', 'o', 'rọ', ',', 'to', 'ubọọn̄', 'n', 'yi', 'e', 'ji', ',', 'o', 'gwu', 'o', '̀', 'bo', 'ki', 'kpulu', 'e', 'ji', ',', 'mè', 'i', 'ki', 'sibi', 'o', 'yerebet', 'i', 'nyi', 'e', 'ji', 'kubọk', 'i', '̂', 'kup', 'be', 'me', 'o', 'REDUP', 'tuuk', 'o', 'REDUP', 'fi', 'i', 'do', 'geelek', '.', '”']
mèinitumuinyisamienibe,“kpọ,mgbọkeyiowuoraikan.nsabọnebirieen̄kwun̄sikpekupkubọkowuokupbe.eyaorọ,toubọọn̄nyieji,ogwuòbokikpulueji,mèikisibioyerebetinyiejikubọkîkupbemeotutuukofifiidogeelek.”
mèinitumuinyisamienibe,“kpọ,m

In [8]:
print(len(train['Obolo']), len(test['Obolo']), len(train['Obolo'])+len(test['Obolo']))

27987 3110 31097


In [219]:
from transformers import AutoTokenizer
bpe_gpt2 = AutoTokenizer.from_pretrained('gpt2', padding_side='left')

In [220]:
bpe_gpt2.vocab_size

50257

In [221]:
print(list(bpe_gpt2.vocab)[:10])

['Ġshoes', 'Mexico', 'Ġinaction', 'asc', 'Ġshared', 'ĠTrack', 'Ġsubord', 'modern', 'Ġtubes', 'Ïī']


In [244]:
tokens = bpe_gpt2.tokenize('Two fighters in the UFC were striking each other, then one of them ended up taking his opponent down and submitting him.')
print(tokens)
ids = bpe_gpt2.encode('Two fighters in the UFC were striking each other, then one of them ended up taking his opponent down and submitting him.')
bpe_gpt2.decode(ids)

['Two', 'Ġfighters', 'Ġin', 'Ġthe', 'ĠUFC', 'Ġwere', 'Ġstriking', 'Ġeach', 'Ġother', ',', 'Ġthen', 'Ġone', 'Ġof', 'Ġthem', 'Ġended', 'Ġup', 'Ġtaking', 'Ġhis', 'Ġopponent', 'Ġdown', 'Ġand', 'Ġsubmitting', 'Ġhim', '.']


'Two fighters in the UFC were striking each other, then one of them ended up taking his opponent down and submitting him.'

In [245]:
sample_words = 'batting disengage archeologists eyewitnesses photographers'
for word in sample_words.split():
    tokens = bpe_gpt2.tokenize(word)
    print(tokens)
    # ids = bpe_gpt2.encode(word)
    # print(bpe_gpt2.decode(ids))

['bat', 'ting']
['dis', 'eng', 'age']
['ar', 'che', 'ologists']
['ey', 'ewitness', 'es']
['phot', 'ographers']


In [255]:
sample_words = 'batting disengage archeologists eyewitnesses photographers'
for word in sample_words.split():
    tokens = eng_tokenizer.encode(word).tokens
    print(tokens)
print(eng_tokenizer.get_vocab_size(), bpe_gpt2.vocab_size)

['Ġbatt', 'ing']
['Ġdis', 'en', 'g', 'age']
['Ġarch', 'eo', 'log', 'ist', 's']
['Ġeyewitnesses']
['Ġp', 'ho', 'to', 'g', 'raph', 'ers']
18627 50257


In [40]:
# tokenizer.save('data/obolo-bpe-tokenizer.json')
# tokenizer = Tokenizer.from_file('data/obolo-bpe-tokenizer.json')

In [217]:
print(obolo_data[100])
out = tokenizer.encode(obolo_data[100])
print(out.tokens)
print(tokenizer.decode(out.ids))

ìkakifieek owot me lek ofiik mgbọ utọọk, mije, ìkilọlọk ofọnti utọọk iso otutuuk ebi uwu kan̄.
['Ġi', 'ÌĢ', 'kakifieek', 'Ġowot', 'Ġme', 'Ġlek', 'Ġofiik', 'Ġmgbo', 'Ì£', 'Ġuto', 'Ì£', 'o', 'Ì£', 'k', ',', 'Ġmije', ',', 'Ġi', 'ÌĢ', 'kilo', 'Ì£', 'lo', 'Ì£', 'k', 'Ġofo', 'Ì£', 'nti', 'Ġuto', 'Ì£', 'o', 'Ì£', 'k', 'Ġiso', 'Ġotutuuk', 'Ġebi', 'Ġuwu', 'Ġkan', 'ÌĦ.']
 ìkakifieek owot me lek ofiik mgbọ utọọk, mije, ìkilọlọk ofọnti utọọk iso otutuuk ebi uwu kan̄.


In [4]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file='data/obolo-bpe-tokenizer.json')
fast_tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=15866, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [54]:
ids = fast_tokenizer(["mè ekekpulu me egwe mè eririeen̄, inyi ekeche utoon̄ esan̄a me lek udun. awaji okpọkpọ, mè imun̄ ibe ke inu cha îjaan̄."], add_special_tokens=True)
print(ids.get('input_ids'))
print(fast_tokenizer.convert_tokens_to_ids('[UNK]'))
fast_tokenizer.batch_decode(ids['input_ids'])

[[56, 55, 3255, 56, 687, 56, 55, 750, 118, 104, 7992, 709, 53, 1158, 53, 16, 56, 95, 425, 10, 156, 526, 50, 152, 191, 56, 55, 470, 53, 108, 87, 109, 177, 49, 81, 587, 149]]
0


[' mè ekekpulu me egwe mè eririeen̄, inyi ekeche utoon̄ esan̄a me lek udun. awaji okpọkpọ, mè imun̄ ibe ke inu cha îjaan̄.']

In [72]:
auto_tk.batch_encode_plus(obolo_data[:5])['input_ids'][0]

[140,
 3,
 23,
 346,
 15719,
 6,
 3,
 9,
 210,
 17815,
 1889,
 3522,
 19,
 77,
 63,
 2,
 29,
 2,
 3,
 51,
 5115,
 3,
 40,
 77,
 63,
 2,
 29,
 2,
 5,
 1]